### Frequency map generation steps

#1. Create date time seried data of events(start and end date for events)
#1a. For flood the data of events are manually created 
2. Read the manually created excel sheet with geoboundary code for each event
3. Merge two dataframe and create year wise fequecny plots of geoboundary code 

In [32]:
import pandas as pd
from datetime import datetime
import numpy as np
import pandas as pd
from ast import literal_eval

filepath='./'

filename=f'{filepath}em_dat_ea_drought_adm2_v2.xlsx'

dfb_dr=pd.read_excel(filename, engine='openpyxl')
#dfb_dr['year']= dfb_dr['Dis No'].str[:4]
#dfb_dr

dfb_dr['adm2_list'] = dfb_dr.geob_adm2_list.apply(lambda x: literal_eval(str(x)))
#dfb_dr['disno_list'] = dfb_dr['Dis No'].apply(lambda x: literal_eval(str(x)))

dfb_dr1=dfb_dr[['Dis No','adm2_list']]

dfb_dr2=dfb_dr1.explode('adm2_list')
dfb_dr2['year']=dfb_dr2['Dis No'].str.split('-').str[0]
#dfb_dr3=dfb_dr2.groupby('adm2_list').agg({'Dis No':lambda x: list(x)})

df = dfb_dr2.groupby(['adm2_list']).agg([ 'count'])

df1=df.reset_index()
df1.columns=['shapeID','count1','count2']
df1['count1'].max()

12

In [26]:
import cartopy.crs as ccrs
import geopandas as gp
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader


adm2=gp.read_file('../../ea_shapefiles/ea_adm2_geoboundaries.shp')

db=pd.merge(adm2,df1,on='shapeID',how='right')

#adm2
db

,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry,count1,count2
0,Alaili Dadda,None,DJI-ADM2-3_0_0-B1,DJI,ADM2,"POLYGON ((43.25550 12.51699, 42.93603 12.29450...",4,4
1,Ali Sabieh,None,DJI-ADM2-3_0_0-B10,DJI,ADM2,"POLYGON ((43.19357 11.37009, 43.00000 11.05495...",6,6
2,As Eyla,None,DJI-ADM2-3_0_0-B11,DJI,ADM2,"POLYGON ((42.30005 11.24280, 42.32147 11.07383...",3,3
3,Dorra,None,DJI-ADM2-3_0_0-B2,DJI,ADM2,"POLYGON ((42.70948 12.37545, 42.70948 12.34583...",4,4
4,Obock,None,DJI-ADM2-3_0_0-B3,DJI,ADM2,"POLYGON ((43.06891 11.83010, 43.06232 11.83672...",6,6
...,...,...,...,...,...,...,...,...
686,Ikungi,None,TZA-ADM2-3_0_0-B92,TZA,ADM2,"POLYGON ((34.59841 -4.61884, 34.59892 -4.61887...",2,2
687,Kahama Township Authority,None,TZA-ADM2-3_0_0-B96,TZA,ADM2,"POLYGON ((32.68160 -3.64886, 32.68931 -3.65776...",3,3
688,Morogoro Urban,None,TZA-ADM2-3_0_0-B97,TZA,ADM2,"POLYGON ((37.65849 -6.69375, 37.67941 -6.69997...",2,2
689,Mtwara Urban,None,TZA-ADM2-3_0_0-B98,TZA,ADM2,"POLYGON ((40.18280 -10.25186, 40.18362 -10.251...",1,1


In [33]:
from matplotlib import colors
from matplotlib import pyplot as plt
import geoplot as gplt

def return_colormap():
    """
    Create colormap of matplotlib based on number of class and given colorcode

    Parameters
    ----------
    params : class object
        Input/Output parameter definitions.
        
    Returns
    -------
    c_cmap : Object
        matplotlib colormap.

    """
    c = matplotlib.colors.ColorConverter().to_rgb
    colorlist=[c("#00c252"), c("#f3ff00"), c("#c85500"), c("#ff0000")]
    color_code=colorlist
    classif= [10, 20, 30, 40]
    c_cmap = LinearSegmentedColormap.from_list("my_colormap",color_code, N=len(classif), gamma=1.0)
    return c_cmap


norm = colors.LogNorm(vmin=db['count1'].min(), vmax=db['count1'].max())



year='1990-2022'
da_plt=plt
plotsize={'width':8.0,'height':9.5}
mainmappost={'x':0.05,'y':-0.05,'width':0.85,'height':1.1}
da_plt.figure(figsize=(plotsize['width'],plotsize['height']))
mainmap=da_plt.axes((mainmappost['x'],mainmappost['y'],mainmappost['width'],mainmappost['height']), projection=ccrs.PlateCarree(),zorder=10)
x_min=21.838949
x_max=51.415695
y_min=-11.745695
y_max=23.145147
mainmap.set_extent([x_min,x_max,y_min,y_max])
mainmap.add_geometries(db['geometry'], ccrs.PlateCarree(),edgecolor='red',facecolor='red', alpha=1,zorder=2)
#mainmap.add_feature(cfeature.BORDERS, linewidth = 2)
mainmap.add_feature(cfeature.OCEAN, facecolor=(0.5,0.5,0.5))
#mainmap.add_feature(cfeature.BORDERS, linewidth = 2)
fname='../../ea_shapefiles/ea_global_background.shp'
mainmap.add_geometries(Reader(fname).geometries(),ccrs.PlateCarree(),facecolor='None')
water_body='../../ea_shapefiles/Africa_Water_Bodies.shp'
mainmap.add_geometries(Reader(water_body).geometries(),ccrs.PlateCarree(),facecolor='grey')
mainmap.text(0.72, 0.03, str(year), fontsize=50,fontweight='bold',ha='left',va='center',color='black', transform = mainmap.transAxes)
output_png_file=f'output/fl/{str(year)}_fl.png'
#da_plt.savefig(output_png_file, transparent=False,bbox_inches='tight')
#da_plt.close()
da_plt.show()

ModuleNotFoundError: No module named 'geoplot'

## Floods

In [20]:
import pandas as pd
from datetime import datetime
import numpy as np
import pandas as pd
from ast import literal_eval

filepath='./'

filename=f'{filepath}em_dat_ea_flood_adm2_v0.xlsx'

dfb_dr=pd.read_excel(filename, engine='openpyxl')
#dfb_dr['year']= dfb_dr['Dis No'].str[:4]
#dfb_dr

dfb_dr['adm2_list'] = dfb_dr.geob_adm2_list.apply(lambda x: literal_eval(str(x)))
#dfb_dr['disno_list'] = dfb_dr['Dis No'].apply(lambda x: literal_eval(str(x)))

dfb_dr1=dfb_dr[['Dis No','adm2_list']]

dfb_dr2=dfb_dr1.explode('adm2_list')
dfb_dr2['year']=dfb_dr2['Dis No'].str.split('-').str[0]
#dfb_dr3=dfb_dr2.groupby('adm2_list').agg({'Dis No':lambda x: list(x)})

df = dfb_dr2.groupby(['adm2_list']).agg([ 'count'])

df1=df.reset_index()
df1.columns=['adm2_list','count1','count2']
df1

,adm2_list,count1,count2
0,ETH-ADM2-3_0_0-B37,1,1
1,ETH-ADM2-3_0_0-B43,1,1
2,KEN-ADM2-3_0_0-B11,2,2
3,KEN-ADM2-3_0_0-B12,2,2
4,KEN-ADM2-3_0_0-B15,1,1
...,...,...,...
721,UGA-ADM2-3_0_0-B95,1,1
722,UGA-ADM2-3_0_0-B96,1,1
723,UGA-ADM2-3_0_0-B97,1,1
724,UGA-ADM2-3_0_0-B98,1,1
